In [1]:
# Configuración inicial
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from omegaconf import OmegaConf
from hydra import initialize, compose

# Configurar PROJECT_ROOT y agregar al PYTHONPATH
PROJECT_ROOT = Path.cwd().parent if "notebooks" in str(Path.cwd()) else Path.cwd()
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

%matplotlib inline

# Inicializar Hydra y cargar configuración
with initialize(version_base=None, config_path="../config"):
    cfg = compose(config_name="main")

# Importar módulos del proyecto
from src.utils.io import read_file
from src.panel.builder import PanelBuilder

In [2]:
# Cargar datos fundamentales
tax_data = read_file(os.path.join(cfg.data.final.base_dir, cfg.data.final.vehicle_tax.file))
gdp_data = read_file(os.path.join(cfg.data.estimated.base_dir, cfg.data.estimated.projected_subnational_gdp.file))
pop_data = read_file(os.path.join(cfg.data.estimated.base_dir, cfg.data.estimated.projected_population.file))

print(tax_data.head)
print(gdp_data.head)
print(pop_data.head)


<bound method NDFrame.head of       DEPARTAMENTO          1990          1991          1992          1993  \
0          Artigas  2.877408e+06  3.344077e+06  3.141777e+06  2.812283e+06   
1        Canelones  9.214381e+06  9.658696e+06  1.189412e+07  1.206996e+07   
2      Cerro Largo  2.219658e+06  2.908989e+06  3.090363e+06  2.983826e+06   
3          Colonia  4.596641e+06  5.456754e+06  6.154163e+06  6.606014e+06   
4          Durazno  1.766631e+06  2.599012e+06  2.802381e+06  2.734942e+06   
5           Flores  1.007930e+06  1.325770e+06  1.527299e+06  1.680290e+06   
6          Florida  2.272781e+06  2.827767e+06  3.430658e+06  3.262789e+06   
7        Lavalleja  1.706129e+06  2.183116e+06  2.789591e+06  2.830792e+06   
8        Maldonado  1.123103e+07  1.467954e+07  2.007499e+07  2.170152e+07   
9       Montevideo  6.614811e+07  6.670958e+07  6.992327e+07  6.945469e+07   
10        Paysandu  3.923681e+06  5.033081e+06  5.697906e+06  5.845809e+06   
11       Rio Negro  2.007020e+06  

In [3]:
from src.panel.inputs import transform_tax_data, transform_gdp_data, transform_pop_data

# Configuraciones con funciones de transformación específicas
tax_config = {
    'transform_func': transform_tax_data,
    'merge_cols': ['año', 'departamento']
}

gdp_config = {
    'transform_func': transform_gdp_data,
    'merge_cols': ['año', 'departamento']
}

pop_config = {
    'transform_func': transform_pop_data,
    'merge_cols': ['año', 'departamento']
}


In [4]:
# Construir panel base
panel = (PanelBuilder()
    .add_variable(tax_data, 'recaudacion', tax_config)
    .add_variable(gdp_data, 'pib', gdp_config)
    .add_variable(pop_data, 'poblacion', pop_config)
    .sort(['departamento', 'año'])
    .build()
)


In [5]:
# Versión alternativa más simple
styled_panel = (panel
    .set_index(['departamento', 'año'])
    .style
    .format({
        'recaudacion': '{:,.0f}',
        'pib': '{:,.0f}',
        'poblacion': '{:,.0f}'
    })
    .background_gradient(
        subset=['recaudacion', 'pib'],
        cmap='YlOrRd'
    )
)

display(styled_panel)